<a href="https://colab.research.google.com/github/AlexAr730/1-PROYECTOKAGGLE/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

replicating local resources


In [2]:
!pip install opendatasets
import opendatasets as od

In [3]:
username = "jhonalexandera"
key = "477f009515fd82380aa7cd216ebfbd69"

In [4]:
dataset_link="https://www.kaggle.com/competitions/udea-ai-4-eng-20251-pruebas-saber-pro-colombia"
od.download(dataset_link)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: jhonalexandera
Your Kaggle Key: ··········


100%|██████████| 29.9M/29.9M [00:00<00:00, 1.10GB/s]


Extracting archive ./udea-ai-4-eng-20251-pruebas-saber-pro-colombia/udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip to ./udea-ai-4-eng-20251-pruebas-saber-pro-colombia


In [5]:
import os
os.chdir("udea-ai-4-eng-20251-pruebas-saber-pro-colombia")
os.listdir()

['test.csv', 'submission_example.csv', 'train.csv']

In [6]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185250 test.csv
   692501  10666231 143732449 train.csv
  1286075  15528571 207634372 total


In [7]:
import pandas as pd
import numpy as np

z = pd.read_csv("train.csv")
print ("shape of loaded dataframe", z.shape)


shape of loaded dataframe (692500, 21)


In [8]:
z.head(3)

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264


In [9]:
z.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 21 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              692500 non-null  int64  
 1   PERIODO                         692500 non-null  int64  
 2   ESTU_PRGM_ACADEMICO             692500 non-null  object 
 3   ESTU_PRGM_DEPARTAMENTO          692500 non-null  object 
 4   ESTU_VALORMATRICULAUNIVERSIDAD  686213 non-null  object 
 5   ESTU_HORASSEMANATRABAJA         661643 non-null  object 
 6   FAMI_ESTRATOVIVIENDA            660363 non-null  object 
 7   FAMI_TIENEINTERNET              665871 non-null  object 
 8   FAMI_EDUCACIONPADRE             669322 non-null  object 
 9   FAMI_TIENELAVADORA              652727 non-null  object 
 10  FAMI_TIENEAUTOMOVIL             648877 non-null  object 
 11  ESTU_PRIVADO_LIBERTAD           692500 non-null  object 
 12  ESTU_PAGOMATRICU

# Preprocesado y limpieza

In [10]:
# Copia del dataframe original
df2 = z.copy()

In [11]:
# 1. Llenar valores nulos

for col in df2.select_dtypes(include='number').columns:
    df2[col] = df2[col].fillna(df2[col].median())

for col in df2.select_dtypes(include='object').columns:
    df2[col] = df2[col].fillna(df2[col].mode()[0])

In [12]:
# 2. Eliminar duplicada 'FAMI_TIENEINTERNET.1' si es igual a 'FAMI_TIENEINTERNET'

if df2['FAMI_TIENEINTERNET'].equals(df2['FAMI_TIENEINTERNET.1']):
    df2.drop(columns=['FAMI_TIENEINTERNET.1'], inplace=True)

In [13]:
# 3. Mapeo informativo basado en el contexto colombiano

valores_matricula_map = {
    'No pagó matrícula': 4,
    'Más de 7 millones': 3,
    'Entre 5.5 millones y menos de 7 millones': 2.5,
    'Entre 4 millones y menos de 5.5 millones': 2,
    'Entre 2.5 millones y menos de 4 millones': 1.5,
    'Entre 1 millón y menos de 2.5 millones': 1,
    'Entre 500 mil y menos de 1 millón': 0.5,
    'Menos de 500 mil': 0.3,
    np.nan: 2  # Imputación neutral
}

df2['ESTU_VALORMATRICULAUNIVERSIDAD'] = df2['ESTU_VALORMATRICULAUNIVERSIDAD'].map(valores_matricula_map)


In [14]:
# 4. Mapeo informativo basado en el contexto colombiano

departamento_map = {
    'BOGOTÁ': 3, 'ANTIOQUIA': 3, 'VALLE': 3,
    'SANTANDER': 2.5, 'ATLANTICO': 2.5, 'CUNDINAMARCA': 2.5, 'BOLIVAR': 2.5, 'CALDAS': 2.5,
    'TOLIMA': 2, 'HUILA': 2, 'RISARALDA': 2, 'QUINDIO': 2, 'BOYACA': 2, 'META': 2, 'NARIÑO': 2, 'CAUCA': 2,
    'CORDOBA': 1.5, 'NORTE SANTANDER': 1.5, 'SUCRE': 1.5, 'MAGDALENA': 1.5, 'CESAR': 1.5,
    'CHOCO': 1, 'PUTUMAYO': 1, 'GUAVIARE': 1, 'VAUPES': 1, 'AMAZONAS': 1, 'SAN ANDRES': 1,
    'ARAUCA': 1, 'CAQUETA': 1, 'CASANARE': 1, 'LA GUAJIRA': 1,
}

df2['ESTU_PRGM_DEPARTAMENTO'] = df2['ESTU_PRGM_DEPARTAMENTO'].map(departamento_map)



In [15]:
# 5. Mapeo informativo basado en el contexto colombiano

horas_trabajo_map = {
    '0': 4,
    'Menos de 10 horas': 3,
    'Entre 11 y 20 horas': 2,
    'Entre 21 y 30 horas': 1,
    'Más de 30 horas': 0
}
df2['ESTU_HORASSEMANATRABAJA'] = df2['ESTU_HORASSEMANATRABAJA'].map(horas_trabajo_map)


In [16]:
# 6. Cambiamos el tipo de estrato a un entero

df2['FAMI_ESTRATOVIVIENDA'] = df2['FAMI_ESTRATOVIVIENDA'].replace('Sin Estrato', 'Estrato 0')
df2['FAMI_ESTRATOVIVIENDA'] = df2['FAMI_ESTRATOVIVIENDA'].str.extract('Estrato (\d)').astype(int)


In [17]:
# 7. Mapeo informativo basado en el contexto colombiano

educacion_map = {
    'Postgrado': 6,
    'Educación profesional completa': 5,
    'Educación profesional incompleta': 4.5,
    'Técnica o tecnológica completa': 4,
    'Técnica o tecnológica incompleta': 3.5,
    'Secundaria (Bachillerato) completa': 3,
    'Secundaria (Bachillerato) incompleta': 2,
    'Primaria completa': 1.5,
    'Primaria incompleta': 1,
    'Ninguno': 0,
    'No sabe': 0,
    'No Aplica': 0
}

df2['FAMI_EDUCACIONPADRE'] = df2['FAMI_EDUCACIONPADRE'].map(educacion_map)
df2['FAMI_EDUCACIONMADRE'] = df2['FAMI_EDUCACIONMADRE'].map(educacion_map)


In [18]:
# 9. Ejemplo de mapeo por áreas (ajusta según tu criterio)

def asignar_puntaje(programa):
    if 'INGENIERIA' in programa:
        return 4.5  # Puntaje alto para ingenierías
    elif 'MEDICINA' in programa or 'ODONTOLOGIA' in programa:
        return 4.7  # Puntaje muy alto para medicina
    elif 'ADMINISTRACION' in programa:
        return 4.0  # Puntaje medio para administración
    elif 'LICENCIATURA' in programa:
        return 3.8  # Puntaje para licenciaturas
    else:
        return 3.5  # Puntaje por defecto

df2['ESTU_PRGM_ACADEMICO'] = df2['ESTU_PRGM_ACADEMICO'].apply(asignar_puntaje)

In [19]:
from sklearn.preprocessing import LabelEncoder

# 10. Seleccionar columnas tipo object (categóricas)
label_cols = df2.select_dtypes(include='object').columns
if 'RENDIMIENTO_GLOBAL' in label_cols:
    label_cols = label_cols.drop('RENDIMIENTO_GLOBAL')

# 11. Codificar columnas categóricas
label_encoders = {}
for col in label_cols:
    le = LabelEncoder()
    df2[col] = le.fit_transform(df2[col].astype(str))
    label_encoders[col] = le

# 12. (Solo si no se ha hecho antes) Codificar la variable objetivo
if df2['RENDIMIENTO_GLOBAL'].dtype == 'object':
    target_encoder = LabelEncoder()
    df2['RENDIMIENTO_GLOBAL'] = target_encoder.fit_transform(df2['RENDIMIENTO_GLOBAL'])

# 13. Separar X e y
X = df2.drop(['ID', 'RENDIMIENTO_GLOBAL'], axis=1)
y = df2['RENDIMIENTO_GLOBAL']

print("Preprocesamiento completo. Shape X:", X.shape)


Preprocesamiento completo. Shape X: (692500, 18)


In [20]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 20 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              692500 non-null  int64  
 1   PERIODO                         692500 non-null  int64  
 2   ESTU_PRGM_ACADEMICO             692500 non-null  float64
 3   ESTU_PRGM_DEPARTAMENTO          692500 non-null  float64
 4   ESTU_VALORMATRICULAUNIVERSIDAD  692500 non-null  float64
 5   ESTU_HORASSEMANATRABAJA         692500 non-null  int64  
 6   FAMI_ESTRATOVIVIENDA            692500 non-null  int64  
 7   FAMI_TIENEINTERNET              692500 non-null  int64  
 8   FAMI_EDUCACIONPADRE             692500 non-null  float64
 9   FAMI_TIENELAVADORA              692500 non-null  int64  
 10  FAMI_TIENEAUTOMOVIL             692500 non-null  int64  
 11  ESTU_PRIVADO_LIBERTAD           692500 non-null  int64  
 12  ESTU_PAGOMATRICU

In [ ]:
df2

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,3.5,3.0,2.5,3,3,1,3.5,1,1,0,0,1,6.0,2,0.322,0.208,0.310,0.267
1,645256,20212,3.5,2.5,1.5,4,3,0,4.0,1,0,0,0,1,3.5,0,0.311,0.215,0.292,0.264
2,308367,20203,3.5,3.0,1.5,0,3,1,3.0,1,0,0,0,0,3.0,0,0.297,0.214,0.305,0.264
3,470353,20195,4.0,2.5,2.0,4,4,1,0.0,1,0,0,0,1,3.0,3,0.485,0.172,0.252,0.190
4,989032,20212,3.5,3.0,1.5,1,3,1,1.5,1,1,0,0,1,1.5,1,0.316,0.232,0.285,0.294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,3.5,1.0,0.5,2,2,1,3.0,1,0,0,1,1,2.0,2,0.237,0.271,0.271,0.311
692496,754213,20212,3.5,1.5,1.5,0,3,1,1.0,1,0,0,0,1,2.0,0,0.314,0.240,0.278,0.260
692497,504185,20183,3.5,3.0,1.0,3,3,1,3.0,1,0,0,1,1,2.0,1,0.286,0.240,0.314,0.287
692498,986620,20195,3.5,2.0,1.5,3,1,0,1.5,0,0,0,1,1,1.5,0,0.132,0.426,0.261,0.328


In [ ]:
X

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,coef_1,coef_2,coef_3,coef_4
0,20212,3.5,3.0,2.5,3,3,1,3.5,1,1,0,0,1,6.0,0.322,0.208,0.310,0.267
1,20212,3.5,2.5,1.5,4,3,0,4.0,1,0,0,0,1,3.5,0.311,0.215,0.292,0.264
2,20203,3.5,3.0,1.5,0,3,1,3.0,1,0,0,0,0,3.0,0.297,0.214,0.305,0.264
3,20195,4.0,2.5,2.0,4,4,1,0.0,1,0,0,0,1,3.0,0.485,0.172,0.252,0.190
4,20212,3.5,3.0,1.5,1,3,1,1.5,1,1,0,0,1,1.5,0.316,0.232,0.285,0.294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,20195,3.5,1.0,0.5,2,2,1,3.0,1,0,0,1,1,2.0,0.237,0.271,0.271,0.311
692496,20212,3.5,1.5,1.5,0,3,1,1.0,1,0,0,0,1,2.0,0.314,0.240,0.278,0.260
692497,20183,3.5,3.0,1.0,3,3,1,3.0,1,0,0,1,1,2.0,0.286,0.240,0.314,0.287
692498,20195,3.5,2.0,1.5,3,1,0,1.5,0,0,0,1,1,1.5,0.132,0.426,0.261,0.328


In [21]:
y

,RENDIMIENTO_GLOBAL
0,2
1,1
2,1
3,0
4,3
...,...
692495,2
692496,1
692497,3
692498,1
